In [1]:
import os
import cmdstanpy
from cmdstanpy import Model, StanFit, cmdstan_path
from cmdstanpy import Model, StanFit
import ujson as json

<h3> 1. Example: Running the model on Men's World Cup 2014 data</h3>

In [2]:
with open('worldcup_2014.data.json', 'r') as fd:
    data_dict = json.load(fd)
data_dict.keys()

# stan_wc = Model(stan_file='../data/worldcup_gelman/worldcup_rep_ag.stan')
# stan_wc.compile()
# stan_wc

# worldcup_fit = stan_wc.sample(data=data_dict, chains=4)
# worldcup_fit.diagnose()
# worldcup_fit.summary()

dict_keys(['I', 'N', 'team_1', 'team_2', 'score_1', 'score_2', 'prior_score'])

In [3]:
with open('countries.data.json', 'r') as fd_country:
    country_dict = json.load(fd_country)
country_dict.keys()

dict_keys(['countries'])

<h3> 2. Women' World Cup Data 2019 </h3>

In [4]:
import pandas as pd
matches = pd.read_csv("538_data/wwc_matches.csv")
forecasts = pd.read_csv("538_data/wwc_forecasts.csv")
print(matches.columns, forecasts.columns)

Index(['date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2',
       'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1',
       'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object') Index(['forecast_timestamp', 'team', 'group', 'spi', 'global_o', 'global_d',
       'sim_wins', 'sim_ties', 'sim_losses', 'sim_goal_diff', 'goals_scored',
       'goals_against', 'group_1', 'group_2', 'group_3', 'group_4',
       'make_round_of_16', 'make_quarters', 'make_semis', 'make_final',
       'win_league', 'timestamp'],
      dtype='object')


In [5]:
#quarter-final info not yet filled in! Wikipedia to the rescue:[0,1,0,1]
matches.score1.iloc[[44, 45, 46, 47]] = [0,1,0,1]
matches.score2.iloc[[44, 45, 46, 47]] = [3,2,2,2]

/Users/nitya/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
len(matches.team2.unique()), len(matches.team1.unique()), len(forecasts.team.unique())

(24, 24, 24)

In [7]:
#spi is the soccer power index: i.e., the "abilities" of the teams
#Getting the scores before the tournament

prior_tournament = forecasts[forecasts['forecast_timestamp']=='2019-05-16 18:22:11 UTC']
countries = prior_tournament[['team', 'spi']].sort_values(by=['spi'], ascending=False)
wwc2019_country = countries['team'].values
print(wwc2019_country)

['USA' 'France' 'Germany' 'Australia' 'Netherlands' 'England' 'Japan'
 'Canada' 'Brazil' 'Sweden' 'Spain' 'New Zealand' 'Norway' 'China'
 'South Korea' 'Italy' 'Nigeria' 'Cameroon' 'South Africa' 'Jamaica'
 'Scotland' 'Thailand' 'Chile' 'Argentina']


In [9]:
#Creating a dictionary and the 'reverse' dictionary which might be useful later

spi_dict = dict(zip(wwc2019_country, list(range(1, len(wwc2019_country)+1))))
spi_dict_r = dict(zip(list(range(1, len(wwc2019_country)+1)), wwc2019_country))

In [10]:
import numpy as np
mean = np.mean(countries['spi'])
std = np.std(countries['spi'])
countries['prior_score'] = [(x - mean)/std for x in countries['spi']]

In [11]:
matches['team1'] = matches['team1'].apply(lambda x: x.split('Women')[0].rstrip())
matches['team2'] = matches['team2'].apply(lambda x: x.split('Women')[0].rstrip())

<h3> 3. Data input for CmdStanPy/Stan </h3>

Data input needs to be in a dictionary format;
Keys in the dictionary are the names of the variables in the data block of the stan model;

In [12]:
N = len(matches)
I = len(wwc2019_country)
mydict = dict({'I': I,
               'N': N,
                'team_1': matches['team1'].values,
               'team_2': matches['team2'].values,
               'score_1': matches['score1'].values,
               'score_2': matches['score2'].values,
              'prior_score': countries['prior_score'].values})

In [14]:
#Replace team names by their rankings;

mydict['team_1'] = [spi_dict.get(x) for x in mydict['team_1']]
mydict['team_2'] = [spi_dict.get(x) for x in mydict['team_2']]

In [15]:
stan_wc = Model(stan_file='worldcup_rep_ag.stan')
stan_wc.compile()
stan_wc

INFO:cmdstanpy:compiling c++
INFO:cmdstanpy:compiled model file: /Users/nitya/Work/test_tutorial/worldcup_rep_ag


Model(name=worldcup_rep_ag,  stan_file="worldcup_rep_ag.stan", exe_file="/Users/nitya/Work/test_tutorial/worldcup_rep_ag")

In [16]:
worldcup_fit = stan_wc.sample(data=mydict, chains=4)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


In [17]:
worldcup_fit.diagnose()


Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory for all transitions.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


'Processing csv files: /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpasvo2h3l/stan-worldcup_rep_ag-draws-1-hqo331cg.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpasvo2h3l/stan-worldcup_rep_ag-draws-2-bkcpyp74.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpasvo2h3l/stan-worldcup_rep_ag-draws-3-nmva70sf.csv, /var/folders/p4/76cck04j075g95k8wbfvw1ch0000gn/T/tmpasvo2h3l/stan-worldcup_rep_ag-draws-4-48yibdif.csv\n\nChecking sampler transitions treedepth.\nTreedepth satisfactory for all transitions.\n\nChecking sampler transitions for divergences.\nNo divergent transitions found.\n\nChecking E-BFMI - sampler transitions HMC potential energy.\nE-BFMI satisfactory for all transitions.\n\nEffective sample size satisfactory.\n\nSplit R-hat values satisfactory all parameters.\n\nProcessing complete, no problems detected.'

In [18]:
worldcup_fit.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-52.245800,0.312725,7.064460,-65.279200,-51.557800,-41.711000,510.310,413.106,1.001470
beta,0.927774,0.005596,0.262979,0.497464,0.932342,1.346170,2208.320,1787.680,1.001450
alpha[1],0.977424,0.015691,0.704174,-0.173532,0.989907,2.111410,2014.030,1630.400,1.000740
alpha[2],0.331074,0.010127,0.647157,-0.731667,0.336461,1.371360,4083.420,3305.610,0.999980
alpha[3],0.419929,0.010363,0.642889,-0.636432,0.419464,1.482490,3848.930,3115.780,1.000110
alpha[4],-0.180631,0.011418,0.658077,-1.235100,-0.193305,0.914442,3322.020,2689.240,1.000240
alpha[5],0.465455,0.011358,0.636959,-0.568909,0.475318,1.490250,3144.890,2545.850,0.999962
alpha[6],0.581047,0.012473,0.714404,-0.634539,0.613000,1.701630,3280.590,2655.700,0.999735
alpha[7],-0.722364,0.013196,0.723756,-1.942340,-0.690351,0.393536,3008.090,2435.100,0.999801


<h3> 4. Interpreting the model fit </h3>


We're replaying all the games and predicting 